In [1]:
# Gravitational Lensing Research
# Program to take a qlens-generated sample chain, derive kappa vs. radius values. No plotting.
# v3: Simultaneously calculate all matter, dark matter only, and baryons.
# This version is for MS2137.

In [2]:
import time
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import timeit
from IPython.core.debugger import set_trace
import datetime
import concurrent.futures as cf
import multiprocessing as mp
import copy

In [3]:
# set the radii values, in arc seconds, log spaced. NOTE THAT THESE MUST MATCH SCRIPT BELOW
rstart = .1
rstop  = 50
nr = 50
radii = np.logspace(np.log10(rstart), np.log10(rstop), nr, endpoint=False)

In [4]:
# for parallel processing
nproc = 1

In [5]:
name = 'MS2137'
suffix = '.cc.v13'
label = name + suffix
# Set your path here, to the "chains_..." directory of interest
#path = '/home/grant/Documents/qlens-beta/' + name + '/chains_' + label
# path = '/home/kea/KEVIN/UCI/Research/gravlensing/' + name + '/chains_' + label
path = '/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/' + name + '/chains_' + label

In [6]:
# now calculated the median and +/- 1-sigma values
def find_bands(p, samples):
    total = [np.sum(samples[:, i + p] * samples[:, 0]) for i in range(nr)]
    sigma_minus  = np.zeros(nr)
    sigma_plus = np.zeros(nr)
    median = np.zeros(nr)
    for i in range(nr):
        runsum = 0.
        sm_flag = sp_flag = m_flag = False
        # sort the table by increasing kappa for the radius in question
        samsort =  samples[samples[:, i + p].argsort()]
        for row in range(len(samsort)):
            runsum += (samsort[row, 0] * samsort[row, i + p]) / total[i]
            if (not sm_flag) and (runsum > 0.158):
                sm_flag = True
                sigma_minus[i] = samsort[row, i + p]
            if (not sp_flag) and (runsum > 0.841):
                sp_flag = True
                sigma_plus[i] = samsort[row, i + p] 
            if (not m_flag) and (runsum > 0.5):
                m_flag = True
                median[i] = samsort[row, i + p]
    return sigma_minus, sigma_plus, median

In [7]:
def ql_script3(i, p, params, tstart, ns):
    # assembles and runs the QLens script, returning three vectors of kappa values
    # Note that the first and pth columns are not parameters.
    # the pth column is the chi square.
    #like,  mvir, c, rc_kpc, q, theta, xc, yc, shear1, shear2, mtot1, mtot2, raw_chisq, chisq = params[0:p] 
    
    like, k0, gamma, a, s, q, theta, xc, yc, shear1, shear2, mtot1, mtot2, raw_chisq, chisq = params[0:p]
    n = 3
    pars0 = ' '.join([str(item) for item in [k0, gamma, n, a, s, q, theta, xc, yc]]) # Halo parameters
    
    pars1 = ' '.join([str(item) for item in [shear1, shear2]]) # Shear parameters

    pars2 = ' '.join([str(item) for item in [mtot1, 15.0, 0.3, 0.92, -17.5, 0, 0]]) # BCG parameters

    pars3 = ' '.join([str(item) for item in [mtot2, 3.84, 0.3, 0.465, -52.12, -9.2, 14.0]]) # GR parameters

    kappaname = 'plotdata/kappadist_temp_' + str(mp.current_process().name) + '.tmp'
    kappaname_dmo = 'plotdata/kappadist_temp_' + str(mp.current_process().name) + '.dmo.tmp'

    script_part_a1 = '''
    # skeleton script to load bestfit results
    zlens=0.314     # REVIEW
    zsrc=2.0
    zsrc_ref=2.0
    fit method multinest
    shear_components on
    major_axis_along_y off    
    central_image on  # REVIEW
    galsubgrid on
    n_livepts = 4000 
    chisqmag on 
    chisqtol 1e-6 
    chisq_time_delays off 
    analytic_bestfit_src on	
    imgplane_chisq off
    sci_notation off
    chisq_imgsep_threshold = 1e-3  
    nimg_penalty off                 
    chisq_mag_threshold = 0.1  
    analytic_bestfit_src on	
    # gridtype cartesian
    # grid -70 65 -45 55
    imgdata read ../MS2137_SL_Data_V4.dat   #REVIEW
    '''

    script_part_a2 = '''
    lens clear
    lens corecusp '''
    # here we insert the parameters for the first lens (halo)

    script_part_b1 =''' shear='''
    # here we insert the paramters for the shear
    
    script_part_b2 ='''
    lens pjaffe pmode=2 '''
    # here we insert the parameters for the second lens (BCG)

    script_part_b3 ='''
    lens pjaffe pmode=2 '''
    # here we insert the params for anchor galaxy, assuming there is one. If not, comment out "lens pjaffe pmode=2"

    script_part_c = '''
    # no other perturbers for this one. 
    '''
    # Other perturbers, if any, can go in part c above.
    
    script_part_d = '''
    plotkappa ''' + str(rstart) + ' ' + str(rstop) + ' ' + str(nr) + ' '

    
    script = script_part_a1 + script_part_a2 + pars0 + script_part_b1 + pars1\
        + script_part_b2 + pars2 + script_part_b3 + pars3  \
        + script_part_c + script_part_d + kappaname + ' \n' \
        + script_part_d + kappaname_dmo + ' lens=0\n'
    sc_name = 'plotdata/kappascript_' + str(mp.current_process().name) + '.in'

    with open(sc_name, 'w') as outfile:
        outfile.write(script)

    # Now, run the script
    os.system('qlens ' + sc_name + ' -q')
    # read in kappa file back in, and store results
    kdat = np.loadtxt(kappaname)
    kappa = kdat[:, 1]
    kdat_dmo = np.loadtxt(kappaname_dmo)
    kappa_dmo = kdat_dmo[:, 1]
    kappa_bary = kappa - kappa_dmo
    if i==0:
        rad = kdat[:, 0]
        assert np.all(np.abs((rad - radii) / radii) < .001)
    if i%50 == 1:
        tavg = (timeit.default_timer() - tstart) / (i+1.)
        proj_fin =  tavg * (ns -i -1)
        sys.stdout.write("\r{0:5.2%} complete. Avg time per iteration: {1:8.6f} sec. Projected finish in {2:4.2f} min. "\
                   .format((float(i) / ns), tavg,  proj_fin/60))
    sys.stdout.flush() # required in multiprocessing or else it buffers this output
    return kappa, kappa_dmo, kappa_bary

In [8]:
# Function to load chain, create the script, run it, find the bands, and return the new samples and bands.
def do_script(label, path, dmo= False): # the path should be the /chains_... directory
    os.chdir(path)
    print(os.getcwd())
    # Import the chain
    samples = np.loadtxt(path + '/' + label, comments="#", delimiter=None, unpack=False)
    with open(path + '/' + label + '.paramnames') as afile:
        paramnames = afile.readlines()
    paramnames = [item[:-1] for item in paramnames]  # Strip off the \n character.
    samples = np.array(samples)
    ns, p = samples.shape
    print(samples.shape)
    print(paramnames)

    # extend the samples to have nr additional columns
    samples2 = np.append(samples, np.zeros((ns, nr)), axis=1)
    samples2.shape
    # delete all samples with zero weight
    mask = [False if sam[0]==0. else True for sam in samples2]
    samples2 = samples2[mask]
    print(samples2.shape)
    ns = samples2.shape[0]
    sort = samples2[samples2[:,0].argsort()]  # This sorts the samples in ascending order, based on weight (first col.)
 
    # Make duplicates for the dark matter only and baryons samples.
    samples2_dmo = copy.deepcopy(samples2)
    samples2_bary = copy.deepcopy(samples2)
    
    # make sure there is a directory called plotdata. If not, create it.
    if not os.path.exists(os.getcwd()+'/plotdata'):
        os.makedirs(os.getcwd()+'/plotdata')
    
    results = []
    tstart = timeit.default_timer()
    with cf.ProcessPoolExecutor(nproc) as pp:
        for i, params in enumerate(samples2[:, 0:p]):
            results.append((i, pp.submit(ql_script3, i, p, params, tstart, ns)))

    # code waits here until all processes are finished
    sys.stdout.write('\r                                                                                      ')
    print("\nKappa calculation completed in ", np.round((timeit.default_timer() - tstart) / 60, 2), " minutes.")
    
    old_i = -1
    for (i,k) in results:
        samples2[i, p:], samples2_dmo[i, p:], samples2_bary[i, p:] = k.result()
        if old_i +1 != i:
            print('WARNING: Out of order at i=', i)
        old_i = i
      
    sm, sp, med = find_bands(p, samples2)
    sm_dmo, sp_dmo, med_dmo = find_bands(p, samples2_dmo)
    sm_bary, sp_bary, med_bary = find_bands(p, samples2_bary)
    return sm, sp, med, sm_dmo, sp_dmo, med_dmo, sm_bary, sp_bary, med_bary
    

In [9]:
sm, sp, med, sm_dmo, sp_dmo, med_dmo, sm_bary, sp_bary, med_bary = do_script(label, path)

/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/MS2137/chains_MS2137.cc.v13
(111174, 15)
['k0', 'gamma', 'a', 's', 'q', 'theta', 'xc', 'yc', 'shear1', 'shear2', 'mtot1', 'mtot2', 'raw_chisq']
(111174, 65)
                                                                                      
Kappa calculation completed in  108.0  minutes.


In [10]:
np.savez(label + '.all.bands', radii, sm, sp, med)
np.savez(label + '.dmo.bands', radii, sm_dmo, sp_dmo, med_dmo)
np.savez(label + '.bary.bands', radii, sm_bary, sp_bary, med_bary)
print('Results saved to bands file.')  

Results saved to bands file.


In [11]:
# Use Notebook "Kappa band plotter v2" to plot the results.